In [0]:
from IPython.display import clear_output
from google.colab import drive
import os
import numpy as np
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/solemn-ai/finnet/neibhors')
!pip install catboost
clear_output()
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [0]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder, StandardScaler
from catboost import CatBoostClassifier
import networkx as nx
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt

In [0]:
# fix seeds
s = 42
from numpy.random import seed
seed(s)

## Загрузим данные

In [4]:
%%time
edges = pd.read_csv('../data/edges.csv')
ids = pd.read_csv('../data/ids.csv')
vertices = pd.read_csv('../data/vertices.csv')

CPU times: user 2.25 s, sys: 243 ms, total: 2.49 s
Wall time: 2.66 s


In [0]:
inv_edges = edges[['id_2', 'id_1'] + edges.columns[2:].to_list()]
inv_edges.rename({'id_2': 'id_1', 'id_1': 'id_2'}, axis=1, inplace=True)
bi_edges = pd.concat([edges, inv_edges])

In [0]:
model = KeyedVectors.load_word2vec_format("../data/node2vec.bin")

if 'embeddings.csv' not in os.listdir("../data/"):
    embs = pd.DataFrame(np.zeros((len(vertices), model.vector_size + 1)), columns=['id'] + [f"e_{i}" for i in range(model.vector_size)])

    for i in tqdm_notebook(range(1, 1534750)):
        embs.iloc[i-1, 0] = i
        if str(i) in model:
            embs.iloc[i-1, 1:] = model[str(i)]
    embs.to_csv("../data/embeddings.csv", index=False)
    

else:
    embs = pd.read_csv("../data/embeddings.csv")

emb_features = list(embs.columns[embs.columns.str.startswith('e_')])

## Обучим модель

In [0]:
result = pd.DataFrame(columns=['id_1', 'id_2', 'similarity'])

In [8]:
for i in tqdm_notebook(ids["id"]):
    if len(test):
    train = bi_edges[bi_edges['id_1'] == i].sort_values(by='n_transactions', ascending=False).head(501)

    sims = model.most_similar(), topn=2)
    result = result.append(pd.Series((str(i),) + sims[0], index=result.columns), ignore_index=True,)
    result = result.append(pd.Series((str(i),) + sims[1], index=result.columns), ignore_index=True,)

In [9]:
result

,id_1,id_2,similarity
0,524354,324632,0.937389
1,524354,656751,0.900091
2,524354,728013,0.886273
3,524354,450272,0.871050
4,524354,717601,0.924957
...,...,...,...
100195,1244877,649505,0.878823
100196,1244877,1044451,0.899913
100197,1244877,12115,0.893519
100198,1244877,260236,0.830107


## Результат готов к отправке

In [0]:
def drop_same_edges(res):
    res = nx.to_pandas_edgelist(nx.from_pandas_edgelist(res, 'id_1', 'id_2', 'similarity')).rename({'source':'id_1', 'target':'id_2'}, axis=1)
    return res

In [0]:
def write_sub(res, fp='subs/fuckthisfuckthat.csv'):
    res.sort_values(by='similarity', ascending=False)[['id_1', 'id_2']].head(100000).to_csv(fp, index=False)

In [12]:
result

,id_1,id_2,similarity
0,524354,324632,0.937389
1,524354,656751,0.900091
2,524354,728013,0.886273
3,524354,450272,0.871050
4,524354,717601,0.924957
...,...,...,...
100195,1244877,649505,0.878823
100196,1244877,1044451,0.899913
100197,1244877,12115,0.893519
100198,1244877,260236,0.830107


In [0]:
result = drop_same_edges(result)

In [0]:
write_sub(result, 'subs/neigh.csv')

In [4]:
pd.read_csv('subs/neigh.csv')

,id_1,id_2
0,524354,496639
1,206473,551850
2,127685,88070
3,524354,1263617
4,713665,221568
...,...,...
87036,831789,552709
87037,1439287,10254
87038,89473,1353715
87039,1227912,34441
